# TP MODELISATION DU CHURN

# Sommaire

1. [Import de la table préparée et Séparation Train / Test](#sect1)
2. [Standardisation des variables](#sect2)
3. [Modélisation](#sectmodel)
  - [Régression pénalisée](#sect41)
  - [Régression pénalisée (Elastic Net)](#sect42) **BONUS**
  - [Random Forest](#sect43)
  - [GBM (Gradient Boosting Machine)](#sect44)

In [ ]:
import pandas as pd
import os
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score,roc_auc_score,roc_curve,auc

import matplotlib.pyplot as plt

# 1 Chargement des données préparées <a name="sect1" ></a>

In [ ]:
repertoire = "../data"
os.chdir(repertoire)
file=open("churn_prepared.pydata","rb")
data=pickle.load(file)
file.close()

In [ ]:
data.shape

In [ ]:
data.describe()


In [ ]:
data.dtypes

In [ ]:
X= data.drop('Churn?',axis=1)
y=data['Churn?'].astype(int)

In [ ]:
y.value_counts(normalize=True)

#### Construire l'échantillon d'apprentissage et de test (vérifier la représentativité du taux de cible) 


In [ ]:
# Attention il faut respecter la meme selection  test_size=0.3,random_state=42
X_train, X_test, y_train, y_test = ...

# 2. Standardisation des variables <a name="sect2" ></a>

#### Appliquer une étape de standardisation des variables sur le train et le test

In [ ]:
from sklearn.preprocessing import StandardScaler
norm= ...  
X_train = ...
X_test = ...

# 3. Modélisation <a name="sectmodel" ></a>

In [ ]:
# Import de la fonction de lift à utiliser
os.chdir('/home/jupyter/python_ml/modules/')
from fonctions_metrics import lift

## 3.1 PARAMETRER LE MODELE REG PENALISEE <a name="sect41" ></a>
Choisir ridge (L2) ou lasso (L1) ou elasticnet, avec la classe LogisticRegression
Définir une grille d'hyper param pour C
et utliser GridSearchCV pour la balayer

In [ ]:
## Regression logistique avec pénalité lasso et grid search
#### on cherche par CV le meilleur C (1/alpha) le coef de regularisation

In [ ]:
param = [{"C":...}]

# Grid Search : on indique le modele et la grille de param
modeleLassoCV = GridSearchCV (.... n_jobs=-1,scoring='roc_auc')
modeleLassoCV = modeleLassoCV.fit(X_train,y_train)
modeleLassoCV.best_params_

In [ ]:
# Je constate que le meilleur modeles est 0.05 donc je réentraine sur le Train avec les meilleurs paramètres 
model_LogitL1 =LogisticRegression(...)


In [ ]:
# récupérer les coef
coef=list(model_LogitL1.coef_[0]) # c un array2d, 
len(coef) # il y a 20 
coef

In [ ]:
# combien d'élemnts non nuls ?
feature_0= [i for i in coef if i==0] #print(feature_0)
print(len(feature_0))
# PLus on baisse C, plus de colonnes s'annulent (0)

### Analyser les résultats et les performances 

In [ ]:
# Prediction des probabilités de 1 

probas_test = ...
probas_train = ...

In [ ]:
# score: accuracy taux de bien classé global au cutoff de 0.5
print(model_LogitL1.score(X_test, y_test))
print(model_LogitL1.score(X_train,y_train))

In [ ]:
#AUC
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))

In [ ]:
print(lift(probas_train,X_train,y_train))
print(lift(probas_test,X_test,y_test))

# Le lift à 10% : je prend les 10% plus fortes proba de churn (individus les plus risqués),
# taux de churn / taux de churn moyen 

In [ ]:
#compute lift at 5%
print(lift(probas_train,X_train,y_train,p=5))
print(lift(probas_test,X_test,y_test,p=5))

In [ ]:
# ENR LES RESULTATS DANS UN DICTIONNAIRE POUR POUVOIR COMPARER
model='Score Lasso'
# métriques (liste de dictionnaires)
metriques = [{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}]
metriques

In [ ]:
# mat de confusion à un cutoff donné
pred_test = (probas_test > 0.14).astype(int)
print(confusion_matrix(y_test, pred_test)
print(pd.crosstab(y_test,pred_test, normalize="index"))
print(pd.crosstab(y_test,pred_test, normalize="columns"))

#### Afficher le classification report entre un cutoff donnée (ex : 0,14 et le cutoff par défaut)

In [ ]:
target_names = ['Fidèles','Churners']

# métriques au cutoff donné
print(classification_report(... target_names=target_names))
# métriques au cutoff par défaut
print(classification_report(..., target_names=target_names))



## 3.2 PARAMETRER LE MODELE REG PENALISEE ELASTICNET <a name="sect42" ></a>

#### Disponible dans le notebook de correction

## 3.3 PARAMETRER LE MODELE RANDOMFOREST <a name="sect43" ></a>
Tester un modle light peu profond et un modele complexe

In [ ]:
rf_light = RandomForestClassifier(  ...
                                     )    


In [ ]:
rf_complex= RandomForestClassifier(  ...
                                     )    

In [ ]:
# Apprentissage des modeles sur le train


### Analyser les résultats et les performances

In [ ]:
# Prediction des probabilités de 1 
probas_test = ...
probas_train = ...

In [ ]:
#AUC sur train et test avec roc_auc_score
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))
# un peu de surapprentissage mais peu ce qui est normal car ce sont des arbres peu profonds

In [ ]:
print(lift(probas_train,X_train,y_train))
print(lift(probas_test,X_test,y_test))

In [ ]:
# AJOUTE LES RESULTATS
model='rf light'
# métriques (liste de dictionnaires)
metriques.append([{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}])
metriques

## 3.4 PARAMETRER LE MODELE GRADIENT BOOSTING MACHINE <a name="sect44" ></a>


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbt_noRand05=GradientBoostingClassifier(loss='deviance', learning_rate=0.05,
                           n_estimators=500,
                           subsample=1.0, min_samples_split=20, min_samples_leaf=10,
                           max_depth=4,
                                       random_state=42)

In [ ]:
# Apprentissage du modele
gbt_noRand05.fit(X_train,y_train)

Analyse du nb iterations optimal en affichant la fonction de perte sur le test

In [ ]:
niter=500
iter = np.arange(niter) + 1
test_deviance = np.zeros((niter,), dtype=np.float64)
# staged_decision_functio : décision fonction à chaque iteration
for i, y_pred in enumerate(gbt_noRand05.staged_decision_function(X_test)):
    # clf.loss_ assumes that y_test[i] in {0, 1}
    test_deviance[i] = gbt_noRand05.loss_(y_test, y_pred)

plt.figure(figsize=(8, 6))
# Erreur sur le test (evolution deviance)
plt.plot(iter,test_deviance,label='Test',color='darkorange')
        # min vers 100 
# Erreur sur apprentissage (evolution deviance)
plt.plot(iter,gbt_noRand05.train_score_,label='Apprentissage',color='navy')    
# Diminution de l'erreur rapport modele precedant (par rapport au oob)
#plt.plot(iter,gbt_noRand05.oob_improvement_)
plt.legend(loc="upper right", fontsize=12)


In [ ]:
# algo avec randomisation subsample=0.5 et max_feature = racine(nb variable)

gbt_Rand=GradientBoostingClassifier(loss='deviance', learning_rate=0.05,
                           n_estimators=500,
                           subsample=0.5, min_samples_split=30, min_samples_leaf=20,
                           min_weight_fraction_leaf=0.005,
                           max_depth=3,max_leaf_nodes=12,max_features="sqrt", random_state=42)
# Apprentissage du modele
gbt_Rand.fit(X_train,y_train)

In [ ]:
niter=500
iter = np.arange(niter) + 1
test_deviance = np.zeros((niter,), dtype=np.float64)
# staged_decision_functio : décision fonction à chaque iteration
for i, y_pred in enumerate(gbt_Rand.staged_decision_function(X_test)):
    # clf.loss_ assumes that y_test[i] in {0, 1}
    test_deviance[i] = gbt_Rand.loss_(y_test, y_pred)

plt.figure(figsize=(8, 6))
# Erreur sur le test (evolution deviance)
plt.plot(iter,test_deviance,label='Test',color='darkorange')
        # min vers 100 
# Erreur sur apprentissage (evolution deviance)
plt.plot(iter,gbt_Rand.train_score_,label='Apprentissage',color='navy')    
# Diminution de l'erreur rapport modele precedant (par rapport au oob)
#plt.plot(iter,gbt_noRand05.oob_improvement_)
plt.legend(loc="upper right", fontsize=12)

In [ ]:
# algo early stopping (150 itération)

gbt_Rand150=GradientBoostingClassifier(loss='deviance', learning_rate=0.05,
                           n_estimators=150,
                           subsample=0.5, min_samples_split=30, min_samples_leaf=20,
                           min_weight_fraction_leaf=0.005,
                           max_depth=3,max_leaf_nodes=12,max_features="sqrt", random_state=42)
# Apprentissage du modele
gbt_Rand150.fit(X_train,y_train)

In [ ]:
niter=150
iter = np.arange(niter) + 1
test_deviance = np.zeros((niter,), dtype=np.float64)
# staged_decision_functio : décision fonction à chaque iteration
for i, y_pred in enumerate(gbt_Rand150.staged_decision_function(X_test)):
    # clf.loss_ assumes that y_test[i] in {0, 1}
    test_deviance[i] = gbt_Rand150.loss_(y_test, y_pred)

plt.figure(figsize=(8, 6))
# Erreur sur le test (evolution deviance)
plt.plot(iter,test_deviance,label='Test',color='darkorange')
        # min vers 100 
# Erreur sur apprentissage (evolution deviance)
plt.plot(iter,gbt_Rand150.train_score_,label='Apprentissage',color='navy')    
# Diminution de l'erreur rapport modele precedant (par rapport au oob)
#plt.plot(iter,gbt_noRand05.oob_improvement_)
plt.legend(loc="upper right", fontsize=12)

### Analyser les résultats et les performances (light) 

In [ ]:
# Prediction des probabilités de 1 , array2d
probas_test = gbt_noRand05.predict_proba(X_test)[:,1]
probas_train = gbt_noRand05.predict_proba(X_train)[:,1]


In [ ]:
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))


In [ ]:
# AJOUTE LES RESULTATS
model='gbm no rand'
# métriques (liste de dictionnaires)
metriques.append([{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}])
metriques
# conclusion  le modele RF est plus predictif mais moins confiance dans sa capacité à etre robuste à moyen terme


In [ ]:
# Prediction des probabilités de 1 , array2d
probas_test = gbt_Rand.predict_proba(X_test)[:,1]
probas_train = gbt_Rand.predict_proba(X_train)[:,1]


In [ ]:
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))


In [ ]:
# AJOUTE LES RESULTATS
model='gbm rand'
# métriques (liste de dictionnaires)
metriques.append([{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}])
metriques
# conclusion  le modele RF est plus predictif mais moins confiance dans sa capacité à etre robuste à moyen terme


In [ ]:
# Prediction des probabilités de 1 , array2d
probas_test = gbt_Rand150.predict_proba(X_test)[:,1]
probas_train = gbt_Rand150.predict_proba(X_train)[:,1]


In [ ]:
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))


In [ ]:
# AJOUTE LES RESULTATS
model='gbm rand 150'
# métriques (liste de dictionnaires)
metriques.append([{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}])
metriques
# conclusion  le modele RF est plus predictif mais moins confiance dans sa capacité à etre robuste à moyen terme
